<a href="https://colab.research.google.com/github/DanielaManate/SentimentAnalysis-TopicModeling/blob/master/SA5_Sentiment_Analysis_Lexicon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install afinn

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp36-none-any.whl size=53452 sha256=9f1a4cd4d1ceca5f45b1f2bf661473366ce871df9da7db3c1be1b212de5d12fc
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn


# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

from textblob import TextBlob

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

from afinn import Afinn

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
path = 'gdrive/My Drive/GH_NLP_Course/Data/'

# Reading Data

In [4]:
# Reading the dataset
reviews = pd.read_csv(path + 'Input/3.input_data_prepped_bow.csv')
reviews.head(2)

,rest_id,text,rating,char_count,positive,text_prep,text_prep_tokens,word_len_prep,text_prep_lim,text_prep_lim_tokens,word_len_prep_lim
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4,302,1,arrived reservation pm seat -PRON- right time ...,"['arrived', 'reservation', 'pm', 'seat', '-PRO...",27,arrived reservation pm seat right time restura...,"['arrived', 'reservation', 'pm', 'seat', 'righ...",25
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5,111,1,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",10,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",9


# TextBlob

In [5]:
TextBlob('This restaurant has OK food').sentiment

Sentiment(polarity=0.5, subjectivity=0.5)

In [6]:
# Polarity increased to 1
TextBlob('This restaurant has OK food!!!!').sentiment

Sentiment(polarity=1.0, subjectivity=0.5)

In [7]:
# Polarity is lower than before, even if text is more positive
TextBlob('This restaurant has OK food. It is the best in the neighbourhood').sentiment

Sentiment(polarity=0.75, subjectivity=0.4)

In [8]:
# Look at subjectivity
TextBlob('Python is a programming language.').sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [9]:
reviews['pred_textblob_score'] = reviews['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
reviews.head(1)

,rest_id,text,rating,char_count,positive,text_prep,text_prep_tokens,word_len_prep,text_prep_lim,text_prep_lim_tokens,word_len_prep_lim,pred_textblob_score
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4,302,1,arrived reservation pm seat -PRON- right time ...,"['arrived', 'reservation', 'pm', 'seat', '-PRO...",27,arrived reservation pm seat right time restura...,"['arrived', 'reservation', 'pm', 'seat', 'righ...",25,0.12627


In [10]:
reviews['pred_textblob_score'].describe()

count    9365.000000
mean        0.247959
std         0.271744
min        -1.000000
25%         0.083333
50%         0.271488
75%         0.426852
max         1.000000
Name: pred_textblob_score, dtype: float64

In [11]:
reviews['pred_textblob'] = np.where(reviews['pred_textblob_score']>=0, 1, 0)
reviews.head(2)

,rest_id,text,rating,char_count,positive,text_prep,text_prep_tokens,word_len_prep,text_prep_lim,text_prep_lim_tokens,word_len_prep_lim,pred_textblob_score,pred_textblob
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4,302,1,arrived reservation pm seat -PRON- right time ...,"['arrived', 'reservation', 'pm', 'seat', '-PRO...",27,arrived reservation pm seat right time restura...,"['arrived', 'reservation', 'pm', 'seat', 'righ...",25,0.126270,1
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5,111,1,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",10,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",9,0.628571,1


In [12]:
reviews.pred_textblob.describe()

count    9365.000000
mean        0.832141
std         0.373761
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: pred_textblob, dtype: float64

In [13]:
print(classification_report(reviews['positive'], reviews['pred_textblob']))

              precision    recall  f1-score   support

           0       0.94      0.52      0.67      2839
           1       0.82      0.98      0.90      6526

    accuracy                           0.84      9365
   macro avg       0.88      0.75      0.78      9365
weighted avg       0.86      0.84      0.83      9365



# VADER

In [14]:
SentimentIntensityAnalyzer().polarity_scores('This restaurant has OK food')

{'compound': 0.4466, 'neg': 0.0, 'neu': 0.577, 'pos': 0.423}

In [15]:
SentimentIntensityAnalyzer().polarity_scores('This restaurant has OK food!!!!')

{'compound': 0.625, 'neg': 0.0, 'neu': 0.494, 'pos': 0.506}

In [16]:
# Polarity increases sequentially, which is good
SentimentIntensityAnalyzer().polarity_scores(
    'This restaurant has OK food. It is the best in the neighbourhood')

{'compound': 0.7983, 'neg': 0.0, 'neu': 0.584, 'pos': 0.416}

In [17]:
SentimentIntensityAnalyzer().polarity_scores(':)')

{'compound': 0.4588, 'neg': 0.0, 'neu': 0.0, 'pos': 1.0}

In [18]:
SentimentIntensityAnalyzer().polarity_scores(':D')

{'compound': 0.5106, 'neg': 0.0, 'neu': 0.0, 'pos': 1.0}

In [19]:
SentimentIntensityAnalyzer().polarity_scores('the food here sux')

{'compound': -0.3612, 'neg': 0.455, 'neu': 0.545, 'pos': 0.0}

In [20]:
SentimentIntensityAnalyzer().polarity_scores('This restaurant is the worst')

{'compound': -0.6249, 'neg': 0.506, 'neu': 0.494, 'pos': 0.0}

In [21]:
SentimentIntensityAnalyzer().polarity_scores('This restaurant is the WORST')

{'compound': -0.7034, 'neg': 0.547, 'neu': 0.453, 'pos': 0.0}

In [22]:
reviews['pred_vader_score'] = reviews['text'].apply(lambda x: 
                                                             SentimentIntensityAnalyzer().polarity_scores(x)['compound'])
reviews.head(1)

,rest_id,text,rating,char_count,positive,text_prep,text_prep_tokens,word_len_prep,text_prep_lim,text_prep_lim_tokens,word_len_prep_lim,pred_textblob_score,pred_textblob,pred_vader_score
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4,302,1,arrived reservation pm seat -PRON- right time ...,"['arrived', 'reservation', 'pm', 'seat', '-PRO...",27,arrived reservation pm seat right time restura...,"['arrived', 'reservation', 'pm', 'seat', 'righ...",25,0.12627,1,0.8954


In [23]:
reviews['pred_vader_score'].describe()

count    9365.000000
mean        0.602372
std         0.592251
min        -0.997600
25%         0.579700
50%         0.908400
75%         0.965300
max         0.999200
Name: pred_vader_score, dtype: float64

In [24]:
reviews['pred_vader'] = np.where(reviews['pred_vader_score']>=0, 1, 0)
reviews.head(2)

,rest_id,text,rating,char_count,positive,text_prep,text_prep_tokens,word_len_prep,text_prep_lim,text_prep_lim_tokens,word_len_prep_lim,pred_textblob_score,pred_textblob,pred_vader_score,pred_vader
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4,302,1,arrived reservation pm seat -PRON- right time ...,"['arrived', 'reservation', 'pm', 'seat', '-PRO...",27,arrived reservation pm seat right time restura...,"['arrived', 'reservation', 'pm', 'seat', 'righ...",25,0.126270,1,0.8954,1
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5,111,1,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",10,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",9,0.628571,1,0.7644,1


In [25]:
reviews.pred_vader.describe()

count    9365.000000
mean        0.830966
std         0.374802
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: pred_vader, dtype: float64

In [26]:
print(classification_report(reviews['positive'], reviews['pred_vader']))

              precision    recall  f1-score   support

           0       0.94      0.52      0.67      2839
           1       0.83      0.99      0.90      6526

    accuracy                           0.85      9365
   macro avg       0.88      0.76      0.79      9365
weighted avg       0.86      0.85      0.83      9365



# AFINN - Homework

In [27]:
Afinn().score('This restaurant has OK food')

0.0

In [28]:
Afinn().score('This restaurant has OK food. It is the best in the neighbourhood')

3.0

In [29]:
Afinn().score('This restaurant has OK food!!!!')

0.0

In [30]:
reviews['pred_afinn_score'] = reviews['text'].apply(lambda x: Afinn().score(x))
reviews.head(1)

,rest_id,text,rating,char_count,positive,text_prep,text_prep_tokens,word_len_prep,text_prep_lim,text_prep_lim_tokens,word_len_prep_lim,pred_textblob_score,pred_textblob,pred_vader_score,pred_vader,pred_afinn_score
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4,302,1,arrived reservation pm seat -PRON- right time ...,"['arrived', 'reservation', 'pm', 'seat', '-PRO...",27,arrived reservation pm seat right time restura...,"['arrived', 'reservation', 'pm', 'seat', 'righ...",25,0.12627,1,0.8954,1,8.0


In [31]:
reviews['pred_afinn_score'].describe()

count    9365.000000
mean        9.311052
std        10.006154
min       -44.000000
25%         3.000000
50%         9.000000
75%        14.000000
max        83.000000
Name: pred_afinn_score, dtype: float64

In [32]:
reviews['pred_afinn'] = np.where(reviews['pred_afinn_score']>=0, 1, 0)
reviews.head(2)

,rest_id,text,rating,char_count,positive,text_prep,text_prep_tokens,word_len_prep,text_prep_lim,text_prep_lim_tokens,word_len_prep_lim,pred_textblob_score,pred_textblob,pred_vader_score,pred_vader,pred_afinn_score,pred_afinn
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4,302,1,arrived reservation pm seat -PRON- right time ...,"['arrived', 'reservation', 'pm', 'seat', '-PRO...",27,arrived reservation pm seat right time restura...,"['arrived', 'reservation', 'pm', 'seat', 'righ...",25,0.126270,1,0.8954,1,8.0,1
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5,111,1,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",10,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",9,0.628571,1,0.7644,1,9.0,1


In [33]:
reviews.pred_afinn.describe()

count    9365.000000
mean        0.861292
std         0.345660
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: pred_afinn, dtype: float64

In [34]:
print(classification_report(reviews['positive'], reviews['pred_afinn']))

              precision    recall  f1-score   support

           0       0.95      0.44      0.60      2839
           1       0.80      0.99      0.89      6526

    accuracy                           0.82      9365
   macro avg       0.88      0.71      0.74      9365
weighted avg       0.85      0.82      0.80      9365

